In [17]:
import torch
from torch import nn
from d2l import torch as d2l

In [18]:
def vgg_block(num_convs, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.LazyConv2d(out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

In [19]:
class VGG(d2l.Classifier):
    def __init__(self, arch, lr=0.1, num_classes=10):
        super().__init__()
        self.save_hyperparameters()
        conv_blks = []
        for (num_convs, out_channels) in arch:
            conv_blks.append(vgg_block(num_convs, out_channels))
        self.net = nn.Sequential(
            *conv_blks, nn.Flatten(),
            nn.LazyLinear(4096), nn.ReLU(), nn.Dropout(0.5),
            nn.LazyLinear(4096), nn.ReLU(), nn.Dropout(0.5),
            nn.LazyLinear(num_classes))
        self.net.apply(d2l.init_cnn)

In [20]:
VGG(arch=((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))).layer_summary(
    (1, 1, 224, 224))

Sequential output shape:	 torch.Size([1, 64, 112, 112])
Sequential output shape:	 torch.Size([1, 128, 56, 56])
Sequential output shape:	 torch.Size([1, 256, 28, 28])
Sequential output shape:	 torch.Size([1, 512, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
Flatten output shape:	 torch.Size([1, 25088])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 10])


In [21]:
model = VGG(arch=((1, 16), (1, 32), (2, 64), (2, 128), (2, 128)), lr=0.01)
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
data = d2l.FashionMNIST(batch_size=128, resize=(224, 224))
model.apply_init([next(iter(data.get_dataloader(True)))[0]], d2l.init_cnn)
trainer.fit(model, data)

OutOfMemoryError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 8.00 GiB total capacity; 1.14 GiB already allocated; 391.04 MiB free; 1.91 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

1. 与AlexNet相比，VGG在计算速度上要慢得多，而且还需要更多的GPU内存。

1.1 比较AlexNet和VGG所需的参数数量。

AlexNet约有6,200万个参数，而VGG（如VGG-16）有约1.38亿个参数。VGG的参数数量显著多于AlexNet，这导致了更高的计算成本和更大的GPU内存需求。

1.2 比较卷积层和全连接层中使用的浮点运算次数。

在卷积层中，浮点运算次数与卷积核大小、输入输出通道数量和输入图像尺寸有关。在全连接层中，浮点运算次数与输入和输出神经元数量有关。在VGG中，卷积层的浮点运算次数要远大于全连接层。

1.3 如何降低全连接层产生的计算成本？

可以通过以下方法降低全连接层的计算成本：
- 减少全连接层的数量。
- 减少全连接层中的神经元数量。
- 使用全局平均池化（Global Average Pooling）代替全连接层，将特征图池化为一个标量，然后进行分类。

2. 在显示与网络各层相关的尺寸时，我们只看到与八个块（加上一些辅助变换）相关的信息，尽管网络有11层。其他三层去哪里了？

显示的八个块指的是VGG网络中的卷积块。剩下的三层是全连接层，通常用于最后的分类任务。在VGG网络的结构中，这些全连接层位于卷积块之后。

3. 使用VGG论文（Simonyan和Zisserman，2014）中的表1构建其他常见模型，如VGG-16或VGG-19。

根据VGG论文中的描述，可以通过堆叠更多的卷积层和全连接层来构建VGG-16和VGG-19。具体来说，VGG-16包括13个卷积层和3个全连接层，而VGG-19包括16个卷积层和3个全连接层。这些模型具有更深的网络结构，因此可以学习到更复杂的特征表示。

4. 将Fashion-MNIST的分辨率从28*28上采样到224*224是非常浪费的。尝试修改网络结构和分辨率转换，例如将输入尺寸改为56或84。在不降低网络准确性的情况下，可以这样做吗？参考VGG论文（Simonyan和Zisserman，2014）以获取在下采样之前添加更多非线性的想法。

可以通过减少网络层数、调整卷积核大小和步长以适应较低分辨率的输入。例如，将输入尺寸调整为56或84，然后相应地调整网络结构。在调整网络时，可以参考VGG论文中的建议，例如在下采样之前添加更多的非线性激活函数。为了保持准确性，可能需要对网络结构进行一些实验和调整。